In [2]:
from DbConnector import DbConnector
from part2 import DbExecutor

In [3]:
# Set up the program
program = DbConnector()
executor = DbExecutor()

Connected to: 8.0.34
You are connected to the database: ('test_db',)
-----------------------------------------------

Connected to: 8.0.34
You are connected to the database: ('test_db',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [3]:
# Get the number of Users
users = executor.get_table_size("User")
print(f"There are {users[0]} users")

# Get the number of Activities
activities = executor.get_table_size("Activity")
print(f"There are {activities[0]} activities")

# Get the number of TrackPoints
trackpoints = executor.get_table_size("TrackPoint")
print(f"There are {trackpoints[0]} trackpoints")


There are (182,) users
There are (7877,) activities
There are (5355109,) trackpoints


### Task 2a

In [4]:
# Get the average number of trackpoints per user
executor.findAvgTrackpoints()


  user_id    AVG(trackpoints)
---------  ------------------
      000            670.871
      001            824.965
      002            924.801
      003            807.387
      004            761.858
      005            587.548
      006            801.125
      007            758.275
      008           1376
      009           1218.32
      010           1107.5
      011            451.756
      012            629.783
      013           1388.3
      014            905.936
      015            806.583
      016            690.222
      017            868.245
      018            540.795
      019            372.291
      020            704.638
      021            475
      022            900.695
      023            979.312
      024            860.408
      025            606.295
      026           1034.71
      027           2037
      028           1195.36
      029           1713.17
      030            871.352
      031            489.5
      032           1190.43
      

In [5]:
# Get the minimum number of trackpoints per user
executor.findMinTrackpoints()


  user_id    MIN(trackpoints)
---------  ------------------
      000                   5
      001                  33
      002                   4
      003                   3
      004                   4
      005                   5
      006                  14
      007                   6
      008                 165
      009                 134
      010                 663
      011                   3
      012                  64
      013                  13
      014                   3
      015                  22
      016                   7
      017                   4
      018                   3
      019                  11
      020                  22
      021                 475
      022                  15
      023                  31
      024                   4
      025                   3
      026                   9
      027                1760
      028                  32
      029                   4
      030                   9
      031 

In [6]:
# Get the maximum number of trackpoints per user
executor.findMaxTrackpoints()


  user_id    MAX(trackpoints)
---------  ------------------
      000                2359
      001                2472
      002                2438
      003                2485
      004                2482
      005                2058
      006                2478
      007                2228
      008                2499
      009                2396
      010                1964
      011                2306
      012                2277
      013                2486
      014                2499
      015                2411
      016                2360
      017                2471
      018                2245
      019                2276
      020                2201
      021                 475
      022                2421
      023                2215
      024                2377
      025                2464
      026                2396
      027                2480
      028                2477
      029                2440
      030                2454
      031 

### Task 4

Find all users who have taken a bus.

In [7]:
# Get the users who have taken the bus
users = executor.get_user_taken_bus()
users = "".join([f'\n user: {user[0]}' for user in users[0]])
print(f"The user ids for user that have taken the bus: {users}")

The user ids for user that have taken the bus: 
 user: 010
 user: 052
 user: 062
 user: 073
 user: 081
 user: 084
 user: 085
 user: 091
 user: 092
 user: 112
 user: 125
 user: 128
 user: 175


### Task 5

In [8]:
executor.findTop10TransportationsUsers()


  user_id    DifferentTransportation
---------  -------------------------
      128                          9
      062                          7
      085                          4
      084                          3
      058                          3
      163                          3
      078                          3
      081                          3
      112                          3
      065                          2


### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [9]:
users = executor.get_user_activity_over_a_day()

num = set()
for user in users[0]:
    num.add(user[0])

count = len(num)

print(f"Number of users with activity that ends the next day: {count}")

Number of users with activity that ends the next day: 66


b) List the transportation mode, user id and duration for these activities.

In [10]:
users = "".join([f'\n user: {user[0]} transportation mode: {user[1]} duration: {user[2]}' for user in users[0]])
print(f"The user who have an end date one day after the start day: {users}")
print(users[0])

The user who have an end date one day after the start day: 
 user: 000 transportation mode: None duration: 4:09:15
 user: 000 transportation mode: None duration: 0:10:45
 user: 000 transportation mode: None duration: 11:03:55
 user: 000 transportation mode: None duration: 1:43:20
 user: 000 transportation mode: None duration: 16:24:06
 user: 001 transportation mode: None duration: 6:54:46
 user: 001 transportation mode: None duration: 0:09:30
 user: 001 transportation mode: None duration: 0:24:57
 user: 001 transportation mode: None duration: 14:17:28
 user: 001 transportation mode: None duration: 8:26:34
 user: 001 transportation mode: None duration: 1:19:41
 user: 001 transportation mode: None duration: 0:55:48
 user: 001 transportation mode: None duration: 0:58:08
 user: 001 transportation mode: None duration: 0:42:03
 user: 001 transportation mode: None duration: 13:30:14
 user: 001 transportation mode: None duration: 2:40:20
 user: 001 transportation mode: None duration: 0:46:26
 

### Task 8

In [6]:
executor.findCloseUsers()



79 have been close to each other in time and space


TypeError: 'int' object is not iterable

### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 11

In [7]:
executor.findInvalidActivities()


  user_id    count(*)
---------  ----------
      000         442
      001         116
      002         295
      003         839
      004        1169
      005         126
      006          48
      007         133
      008          33
      009         114
      011          38
      012         163
      013          68
      014         328
      015         136
      016          60
      017         492
      018          97
      019          65
      020          15
      021           1
      022         188
      023          30
      024          73
      025         424
      026          51
      027          13
      028          90
      029          37
      030         509
      031           7
      032          36
      033           3
      034         240
      035         122
      036         125
      037         279
      038         264
      039         423
      040          50
      041         386
      042         117
      043          72
      044 

In [13]:
# Closing the connection
program.close_connection()


-----------------------------------------------
Connection to 8.0.34 is closed
